Here we will take in a print document such as a PDF or an EPUB. We will then create embeddings for it and use an LLM to answer questions about it.

In [1]:
from tqdm import tqdm
import google.generativeai as palm
from IPython.display import display, Markdown

In [2]:
with open("data/transcript.txt", encoding='utf-8') as f:
    doc = f.read()

### Chunking

In [3]:
from chunker import get_chunks

In [4]:
# Make type suitable for chunker function
docs = []
docs.append(doc)

In [5]:
texts = get_chunks(docs)
texts

['Transcript',
 "0:01\n[Music]\n0:18\n[Music]\n0:20\nhello viewers and welcome to this week's\n0:22\nedition of Economics 101 program that\n0:25\ntakes a closer look at the economy also\n0:28\nin relation with the national\n0:30\ndevelopment strategy one that we\n0:32\ncontinuously track on this program the\n0:34\nmany facets of this economy starts with\n0:36\nthe budgeting uh side of it but uh we\n0:38\ntalk about the production of Commodities\n0:41\nprocessing as well as the manufacturing\n0:44\nthat then follows these are critical\n0:46\ncomponents in sustaining an economy and\n0:49\nthen comes the marketing of goods as\n0:51\nwell as Commodities to the market as you\n0:55\nknow the market varies from local\n0:57\nRegional as well as inter International\n1:00\nmarkets as you know Zimbabwe is rich in\n1:03\nminerals but remains an aggr based\n1:05\neconomy but the question of the day\n1:07\ntoday is are we doing enough to Market\n1:09\nourselves as a nation as Zimbabwe and in\n1:13\n

### Embeddings

In [6]:
from uuid import uuid4
from embeddings_palm import get_palm_embeddings

In [7]:
chunks = []
for text in tqdm(texts):
    
    chunks.append(
        {
            'id': str(uuid4()),
            'values': get_palm_embeddings(text),
            'metadata': {
                'text': text
                }
        }
    )

  0%|          | 0/23 [00:00<?, ?it/s]

100%|██████████| 23/23 [01:22<00:00,  3.61s/it]


## Pinecone

In [8]:
import os
import pinecone

### Credentials

In [9]:
pinecone_api_key = os.getenv('PINECONE_API_KEY')

### Creating an Index

In [10]:

index_name = 'general-rag-idx'

# initialize connection to pinecone
pinecone.init(
    api_key= pinecone_api_key,
    environment="gcp-starter"  # next to API key in console
)

In [11]:
# check if index already exists (it shouldn't if this is first time)
if index_name not in pinecone.list_indexes():
    # if does not exist, create index
    pinecone.create_index(
        index_name,
        dimension=len(chunks[0]['values']),
        metric='dotproduct'
    )
# connect to index
index = pinecone.GRPCIndex(index_name)
# view index stats
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 0}},
 'total_vector_count': 0}

### Populating the Index

In [12]:
index.upsert(vectors=chunks)

upserted_count: 23

### Retrieval

#### Create an Embedding for the Query
This comes in the form of a query vector we will name `xq`.

In [20]:
query = "economics, africa, zimbambwe, marketing, neoliberalism, rules based order"

xq = get_palm_embeddings(query)


#### Query Vector Database

In [21]:
res = index.query(xq, top_k=5, include_metadata=True)
res

{'matches': [{'id': '9a883009-6628-4bfa-9a10-37c17919fe34',
              'metadata': {'text': '0:01\n'
                                   '[Music]\n'
                                   '0:18\n'
                                   '[Music]\n'
                                   '0:20\n'
                                   "hello viewers and welcome to this week's\n"
                                   '0:22\n'
                                   'edition of Economics 101 program that\n'
                                   '0:25\n'
                                   'takes a closer look at the economy also\n'
                                   '0:28\n'
                                   'in relation with the national\n'
                                   '0:30\n'
                                   'development strategy one that we\n'
                                   '0:32\n'
                                   'continuously track on this program the\n'
                                   '0:3

## Retreival Augmented Generation

### Stuff Method
We concatenate the text metadata from the vector store directly with our query. This is simple and it works for small prompts.

In [25]:
new_query = "What does the speaker have to say about their country in terms of economics and development matters?"

In [26]:
# get list of retrieved text
contexts = [item['metadata']['text'] for item in res['matches']]

augmented_query = "\n\n---\n\n".join(contexts)+"\n\n-----\n\n"+new_query
display(Markdown(augmented_query))

0:01
[Music]
0:18
[Music]
0:20
hello viewers and welcome to this week's
0:22
edition of Economics 101 program that
0:25
takes a closer look at the economy also
0:28
in relation with the national
0:30
development strategy one that we
0:32
continuously track on this program the
0:34
many facets of this economy starts with
0:36
the budgeting uh side of it but uh we
0:38
talk about the production of Commodities
0:41
processing as well as the manufacturing
0:44
that then follows these are critical
0:46
components in sustaining an economy and
0:49
then comes the marketing of goods as
0:51
well as Commodities to the market as you
0:55
know the market varies from local
0:57
Regional as well as inter International
1:00
markets as you know Zimbabwe is rich in
1:03
minerals but remains an aggr based
1:05
economy but the question of the day
1:07
today is are we doing enough to Market
1:09
ourselves as a nation as Zimbabwe and in
1:13
studio today we are joined by
1:17
m who is a marketing Enthusiast as well
1:21
as expert as well as a brand manager to
1:24
help us unpack these and other issues
1:27
stabile it's such a pleasure to have you
1:29
in studio today it's a pleasure to be
1:30
here thank you so much for having me
1:32
absolutely absolutely and let's get
1:34
right into it to begin with as a

---

absolutely absolutely and let's get
1:34
right into it to begin with as a
1:36
marketing expert can you say that as a
1:38
country we're doing enough to expose
1:40
ourselves to the world in terms of this
1:44
particular field um that you are in
1:46
there's so many more aspects that we can
1:48
tap into you know especially when I
1:50
think around the digital you know aspect
1:53
of marketing that we're in right now I
1:55
find that we're still so behind and to
1:57
me it's tragic because we don't need to
1:59
be that far behind you know third world
2:01
versus first world for instance but it
2:02
doesn't mean that we have to be as far
2:04
behind as we are there's so many more
2:05
platforms we could be tapping into to
2:07
expose ourselves you know and to just
2:08
build more awareness around what it is
2:10
that we are capable of as zimbabweans
2:12
and what it is that we have as a nation
2:14
absolutely absolutely because uh we are
2:16
talking about a space in which uh
2:18
perhaps not to mean to contrast but uh
2:21
we have ICT Innovations we have steps
2:24
that even the second um dispensation is
2:27
taking with regards um to some of the
2:29
line of marketing um that are being
2:32
taken um by this second Republic um can
2:35
you say that the government as well has
2:38

---

4:32
conversations you know around things
4:34
that are changing the economy and that
4:35
could make it better absolutely so if we
4:37
had the government coming into schools
4:39
and coming into universities and
4:40
speaking publicly about what it is that
4:42
branding is and why it's so important
4:44
and how it's helped to build even the
4:45
economy and the government and these
4:47
brands that are making us better it
4:50
would really do a lot of service I think
4:52
to marketers to children and to the
4:54
growth of of Zimbabwe as a nation
4:56
absolutely speaking from your point of
4:57
view we are talking about a brand
5:00
manager yeah so brand Zimbabwe say
5:02
you're given the Reigns yes um we are
5:05
encompassing or incorporating all these
5:07
brands that are perhaps propelling the N
5:10
Narrative of Zimbabwe at large right and
5:13
we are now giving you the project brand
5:15
Zimbabwe what is the first step that you
5:17
will take with regards to coloring in so
5:20
to speak for Zimbabwe so my first thing
5:23
is to to sit down with the most
5:25
important decision makers and ask them
5:28
what do you feel make brand Zimbabwe
5:30
what are the things that you would like
5:31
us to be known for you know because with
5:33
anything I think with the personality
5:35
with the business you know with the
5:36
family there are certain things that you

---

23:01
manager as well as an expert with
23:04
regards to marketing such a pleasure to
23:06
be having you in studio we look forward
23:07
to having you again thank you so much
23:09
for having me cly it really has been an
23:10
absolute pleasure to be here absolutely
23:12
we wish you all the best and yes uh this
23:14
has been economics 101 a jam-packed
23:16
session with regards to everything
23:18
branding Zimbabwe and also more
23:21
information as well as a knowledge on
23:24
the issue of branding marketing and
23:26
Communications here on economics 101 and
23:30
of course we are coming to you from zbc
23:33
TV from me uh ctly gwendy the crew
23:35
behind the scenes and everybody here at
23:38
our pocket little Studio it is a very
23:41
pleasant viewing and God bless
23:48
[Music]
23:56
Zimbabwe
24:03
[Music]
24:05
he

---

with the business you know with the
5:36
family there are certain things that you
5:37
end up being known for and for some
5:39
reason these things can be negative and
5:41
people tend to focus on the negative so
5:43
with us we want to understand what are
5:45
the important things about Zimbabwe that
5:46
we want to be known for because as a
5:48
brand manager my job is to highlight the
5:50
positives you know so I want to know I
5:52
would start with the top three so
5:53
usually what we do is we do three to
5:54
five you know because they're the
5:56
easiest to remember you don't want a
5:57
long list cuz you can have a large
5:58
laundry list of of what you think you're
6:00
doing well what do we want to be known
6:02
for once we've identified what those
6:03
three things are we're now going to sit
6:05
sit down and get into the strategy of
6:07
how we execute a project of building
6:09
that awareness around those important
6:12
factors absolutely absolutely and uh
6:14
taking a look at that um we'll talk
6:16
about the broadness of marketing and
6:18
perhaps uh how we can particularly uh
6:21
focus on uh some of the things that you
6:23
particularly tackle within uh your space
6:26
and scope as a brand manager as well as
6:28
a marketing specialist but for now we're
6:30
going to take a short break here on
6:31

-----

What does the speaker have to say about their country in terms of economics and development matters?

### Primer Template

In [27]:
# system message to 'prime' the model
primer = f"""
You are provided with a text transcript\
it might or might not contain timestamps for each line\
you are expected to summarize this text into something that someone can read at a glance \
If you are unable to find the transcript or it is blank, say so explicitly\
Report the problem, do not make things up
"""

In [28]:
res = palm.chat(context=primer, messages=augmented_query, temperature=0.7)

In [29]:
display(Markdown(res.last))

The speaker is a marketing expert who believes that Zimbabwe is not doing enough to market itself as a nation. They say that there are many more aspects of marketing that Zimbabwe could tap into, such as digital marketing, to expose itself to the world and build more awareness around its capabilities and resources. They also believe that the government could do more to promote marketing and branding, and that having conversations about these topics in schools and universities would help to educate people about their importance.

The speaker's main point is that Zimbabwe needs to do more to market itself if it wants to grow its economy and develop as a nation. They believe that marketing is essential for success in today's world, and that Zimbabwe is missing out on a lot of opportunities by not doing more to promote itself.